### Task 1: 4-point OCP

In [61]:
import crocoddyl
import pinocchio as pin
import numpy as np
import example_robot_data
import mim_solvers
import pybullet as p
import pybullet_data
import time

In [62]:
robot = example_robot_data.load('talos')
model = robot.model
q0 = model.referenceConfigurations["half_sitting"]
x0 = np.concatenate([q0, np.zeros(model.nv)])
data = model.createData()

from supaero2025.meshcat_viewer_wrapper import MeshcatVisualizer

viz = MeshcatVisualizer(robot, robot.collision_model, robot.visual_model)
viz.display(robot.q0)
viz.loadViewerModel()

viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7034/static/


In [63]:
robot = example_robot_data.load('talos')
robot_model = robot.model
q0 = robot_model.referenceConfigurations["half_sitting"]
x0 = np.concatenate([q0, np.zeros(robot_model.nv)])

In [64]:
rf_id = robot_model.getFrameId("right_sole_link")
lf_id = robot_model.getFrameId("left_sole_link")
lh_id = robot_model.getFrameId("gripper_left_joint")

GOAL_POSITION = np.array([.4, .4, 1])
GOAL_PLACEMENT = pin.SE3(pin.utils.rpyToMatrix(-np.pi, -1.5, 1.5), GOAL_POSITION)

In [65]:
from supaero2025.meshcat_viewer_wrapper import MeshcatVisualizer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)
viz.addBox('world/goal',[.1,.1,.1],[0,1,0,1])
viz.applyConfiguration('world/goal',GOAL_PLACEMENT)

viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7035/static/


In [66]:
state = crocoddyl.StateMultibody(robot_model)
actuation = crocoddyl.ActuationModelFloatingBase(state)

contacts = crocoddyl.ContactModelMultiple(state, actuation.nu)

lf_contact = crocoddyl.ContactModel6D(
    state,
    lf_id,
    pin.SE3.Identity(),
    pin.LOCAL_WORLD_ALIGNED,
    actuation.nu,
    np.array([0,0]),
)
rf_contact = crocoddyl.ContactModel6D(
    state,
    rf_id,
    pin.SE3.Identity(),
    pin.LOCAL_WORLD_ALIGNED,
    actuation.nu,
    np.array([0,0]),
)

contacts.addContact("lf_contact", lf_contact)
contacts.addContact("rf_contact", rf_contact)

costModel = crocoddyl.CostModelSum(state, actuation.nu)

w_hand = np.array([1,1,1,0.0001,0.0001,0.0001])
lh_Mref = pin.SE3(np.eye(3), GOAL_POSITION)
activation_hand = crocoddyl.ActivationModelWeightedQuad(w_hand**2)
lh_cost = crocoddyl.CostModelResidual(
    state,
    activation_hand,
    crocoddyl.ResidualModelFramePlacement(state, lh_id, lh_Mref, actuation.nu),
)
costModel.addCost("lh_goal", lh_cost, 100)

In [67]:
# Adding state and control regularization terms
w_x = np.array([0] * 3 + [10.0] * 3 + [0.01] * (state.nv - 6) + [10] * state.nv)
activation_xreg = crocoddyl.ActivationModelWeightedQuad(w_x**2)
x_reg_cost = crocoddyl.CostModelResidual(
    state, activation_xreg, crocoddyl.ResidualModelState(state, x0, actuation.nu)
)
u_reg_cost = crocoddyl.CostModelResidual(
    state, crocoddyl.ResidualModelControl(state, actuation.nu)
)
costModel.addCost("xReg", x_reg_cost, 1e-3)
costModel.addCost("uReg", u_reg_cost, 1e-4)


In [68]:
# Adding the state limits penalization
x_lb = np.concatenate([state.lb[1 : state.nv + 1], state.lb[-state.nv :]])
x_ub = np.concatenate([state.ub[1 : state.nv + 1], state.ub[-state.nv :]])
activation_xbounds = crocoddyl.ActivationModelQuadraticBarrier(
    crocoddyl.ActivationBounds(x_lb, x_ub)
)
x_bounds = crocoddyl.CostModelResidual(
    state,
    activation_xbounds,
    crocoddyl.ResidualModelState(state, 0 * x0, actuation.nu),
)
costModel.addCost("xBounds", x_bounds, 1.0)


In [69]:
# Adding the friction cone penalization
nsurf, mu = np.identity(3), 0.7
cone = crocoddyl.FrictionCone(nsurf, mu, 4, False)
activation_friction = crocoddyl.ActivationModelQuadraticBarrier(
    crocoddyl.ActivationBounds(cone.lb, cone.ub)
)
lf_friction = crocoddyl.CostModelResidual(
    state,
    activation_friction,
    crocoddyl.ResidualModelContactFrictionCone(state, lf_id, cone, actuation.nu),
)
rf_friction = crocoddyl.CostModelResidual(
    state,
    activation_friction,
    crocoddyl.ResidualModelContactFrictionCone(state, rf_id, cone, actuation.nu),
)
costModel.addCost("lf_friction", lf_friction, 1e1)
costModel.addCost("rf_friction", rf_friction, 1e1)

In [70]:
dmodel = crocoddyl.DifferentialActionModelContactFwdDynamics(
    state, actuation, contacts, costModel
)

T = 100


In [71]:
seq = [crocoddyl.IntegratedActionModelEuler(dmodel, 0.001)] * T \
    + [crocoddyl.IntegratedActionModelEuler(dmodel, 0.0)]
    
# problem = crocoddyl.ShootingProblem(x0,seq0+seq1+seq2+seq3,terminalModels[3])
problem = crocoddyl.ShootingProblem(x0,seq,seq[-1])

In [72]:
solver = mim_solvers.SolverCSQP(problem)
solver.with_callbacks = True 
solver.termination_tolerance = 1e-3         # Termination criteria (KKT residual)
solver.max_qp_iters = 1000                  # Maximum number of QP iteration
solver.eps_abs = 1e-5                       # QP termination absolute criteria, 1e-9 
solver.eps_rel = 0.                         # QP termination absolute criteria
solver.use_filter_line_search = True        # True by default, False = use merit function

solver.solve([],[],1000)  # xs_init,us_init,maxiter

False

In [73]:
viz.viewer.jupyter_cell()

In [74]:
viz.play([x[:robot.model.nq] for x in solver.xs],0.001)